<a href="https://colab.research.google.com/github/akash-kaul/ecosys/blob/master/AlgoLoader_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#User definied parameters
host = "https://covidv3.i.tgcloud.io" #@param {type:"string"}
graphname = "MyGraph" #@param {type:"string"}
username = "tigergraph" #@param {type:"string"}
password = "tigergraph" #@param {type:"string"}
version = "3.0.5" #@param ["3.0.5", "3.0.0", "2.6.2"] {allow-input: true}
useCert = True #@param {type:"boolean"}


from IPython.utils import io
import os
from tqdm import tqdm
with tqdm(total=100) as pbar:
  with io.capture_output() as captured:
    pbar.set_description_str('Installing pyTigerGraph')
    
    !pip install pytigergraph    
    import pyTigerGraph as tg
    pbar.update(25)

    ##Grab_GSQL_Queries_BEGIN##
    pbar.set_description_str('Fetching Algorithms')
    import os
    from glob import glob
    !git clone https://github.com/tigergraph/gsql-graph-algorithms.git
    pbar.update(25)
    os.chdir('/content/gsql-graph-algorithms/algorithms/schema-free')
    script_names = glob('*.gsql')
    pbar.update(25)
    ##Grab_GSQL_Queries_END##

    ##Establish_Connection_BEGIN##
    pbar.set_description_str('Establishing Connection')
    conn = tg.TigerGraphConnection(host=host, graphname=graphname, username=username, password=password, version=version, useCert=useCert)
    secret = conn.createSecret()
    token = conn.getToken(secret, setToken=True)
    # API_Token = conn.getToken(API_Secret, setToken=True, lifetime=None)[0]
    pbar.update(25)
    pbar.set_description_str('Connected')
    ##Establish_Connection_END##
        


In [ ]:
#@title
import ipywidgets as widgets
from IPython.display import display
dropdown = widgets.Dropdown(
    options=script_names,
    description='Algo',
    disabled=False,
    )
display(dropdown)

import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Install Query")
output = widgets.Output()

In [ ]:
#@title
import ipywidgets as widgets
# from IPython.display import clear_output
from IPython.display import display
button = widgets.Button(description="Install Query")
output = widgets.Output()

def on_button_clicked(b):
  import re
  query = []
  query_names = []
  with open (dropdown.value, 'r') as file:
      text = file.readlines()
      for line in text:
          new_line = line
          if 'CREATE QUERY' in line:
              q_name = line.find('CREATE QUERY') + 13
              query_names.append(line[q_name:].split('(')[0].strip())
              index = line.find(')')
              new_line = line[:index +1] + f" FOR GRAPH {graphname}" + line[index+1:]

          query.append(new_line)
          Gsql_query = "".join(query)
  
  for name in query_names:
      Gsql_query += ' INSTALL QUERY ' + name
  print("\nQuery is now Installing... Grab a cup of coffee  (approx ~wait 1-2 mins)")    
  conn.gsql(Gsql_query)
  

  with output:
    # clear_output()
    print('''

     (^\-==-/^)
     >\\ == //<
    :== q''p ==:     _
     .__ qp __.    .' )
      / ^--^ \    /\.'
     /_`    / )  '\/
     (  )  \  |-'-/
     \^^,   |-|--'
    ( `'    |_| )
     \-     |-|/
    (( )^---( )) sk

      ''')
    print("=========================================\nCONGRATULATIONS, YOUR QUERY IS INSTALLED!\n=========================================\n")
    print(Gsql_query)

button.on_click(on_button_clicked)
display(button, output)
